**Project Theme:**
Climate Risk & Disaster Management

**Project Title:**
Landslide Risk Prediction

**Problem Statement:**
Landslides cause significant damage to lives, infrastructure, and the environment. Predicting landslide risk levels (Low, Moderate, High, Very High) based on factors like temperature, humidity, precipitation, soil moisture, and elevation can help mitigate these impacts. Accurate risk assessment supports early warning systems, disaster management, and safer land-use planning.

**Description:**
This project uses a dataset containing environmental measurements such as temperature, humidity, precipitation, soil moisture, and elevation to build machine learning models that predict landslide risk levels. The models will reveal how weather patterns and terrain features influence landslide susceptibility. The insights can improve disaster preparedness, land management, and infrastructure development in vulnerable regions.

*Import necessary libraries*

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

*Load The Dataset*

In [2]:
df = pd.read_csv('/content/regenerated_landslide_risk_dataset.csv')

*Explore The Dataset*

In [3]:
print("Data Head:")
print(df.head())

Data Head:
   Temperature (°C)  Humidity (%)  Precipitation (mm)  Soil Moisture (%)  \
0                17            68                 176                 64   
1                26            33                  65                 24   
2                16            81                  56                 52   
3                25            53                 136                 70   
4                34            77                  23                 37   

   Elevation (m) Landslide Risk Prediction  
0            120                       Low  
1            672                       Low  
2             36                       Low  
3            583                       Low  
4            130                       Low  


In [4]:
print("Data Info:")
print(df.info())

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Temperature (°C)           5000 non-null   int64 
 1   Humidity (%)               5000 non-null   int64 
 2   Precipitation (mm)         5000 non-null   int64 
 3   Soil Moisture (%)          5000 non-null   int64 
 4   Elevation (m)              5000 non-null   int64 
 5   Landslide Risk Prediction  5000 non-null   object
dtypes: int64(5), object(1)
memory usage: 234.5+ KB
None


In [5]:
print("Descriptive Stats:")
print(df.describe())

Descriptive Stats:
       Temperature (°C)  Humidity (%)  Precipitation (mm)  Soil Moisture (%)  \
count       5000.000000   5000.000000         5000.000000        5000.000000   
mean          24.815800     62.614000          123.126400          54.915800   
std            6.052867     19.108533           72.144734          20.345788   
min           15.000000     30.000000            0.000000          20.000000   
25%           20.000000     46.000000           61.000000          37.000000   
50%           25.000000     63.000000          121.000000          55.000000   
75%           30.000000     79.000000          186.000000          72.250000   
max           35.000000     95.000000          250.000000          90.000000   

       Elevation (m)  
count    5000.000000  
mean      503.015800  
std       288.700745  
min         0.000000  
25%       253.000000  
50%       505.000000  
75%       757.000000  
max      1000.000000  


In [6]:
print("Missing Values:")
print(df.isnull().sum())
print("\nTotal Missing Values Count:",df.isnull().sum().sum())

Missing Values:
Temperature (°C)             0
Humidity (%)                 0
Precipitation (mm)           0
Soil Moisture (%)            0
Elevation (m)                0
Landslide Risk Prediction    0
dtype: int64

Total Missing Values Count: 0


*Preprocess Data*

In [7]:
print("Unique values in target column before encoding:", df['Landslide Risk Prediction'].unique())
le = LabelEncoder()
df['Landslide Risk Prediction'] = le.fit_transform(df['Landslide Risk Prediction'])
print("Data Head after preprocessing:")
print(df.head())
print("Unique values in target column after encoding:", df['Landslide Risk Prediction'].unique())

Unique values in target column before encoding: ['Low' 'Moderate' 'High' 'Very High']
Data Head after preprocessing:
   Temperature (°C)  Humidity (%)  Precipitation (mm)  Soil Moisture (%)  \
0                17            68                 176                 64   
1                26            33                  65                 24   
2                16            81                  56                 52   
3                25            53                 136                 70   
4                34            77                  23                 37   

   Elevation (m)  Landslide Risk Prediction  
0            120                          1  
1            672                          1  
2             36                          1  
3            583                          1  
4            130                          1  
Unique values in target column after encoding: [1 2 0 3]
